# preloading

In [1]:
import os
from pathlib import Path
import datetime
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import pandas as pd
import functools
import itertools

import pyflexlab
import pyflexlab.pltconfig.color_preset as colors
from pyflexlab.constants import cm_to_inch, next_lst_gen, constant_generator
from pyflexlab.file_organizer import FileOrganizer
from pyflexlab.data_process import DataProcess
from pyflexlab.data_plot import DataPlot
# MeasureManager module will need the NIDAQmx module to work,
from pyflexlab.measure_manager import MeasureManager

set with PYLAB_DB_LOCAL_DONGKAI
set with PYLAB_DB_OUT_DONGKAI
read from PYLAB_DB_LOCAL:D:\ONEDRIVE-NTU\OneDrive - Nanyang Technological University\script-tools\data_files
read from PYLAB_DB_OUT:D:\ONEDRIVE-NTU\OneDrive - Nanyang Technological University\database


In [2]:
project_name = "Date-Material"
folder = FileOrganizer(project_name)
plotobj = DataPlot(project_name)
#FileOrganizer.reload_paths(local_db_path=od_path / "script-tools" / "data_files", out_db_path=od_path / "database")

In [ ]:
folder.tree()

In [6]:
FileOrganizer.name_fstr_gen("I_source_sweep_dc","V_sense","T_vary")

('I-V-T',
 'Imax{maxi}A-step{stepi}A-{iin}-{iout}-swpmode{mode}_V{note}-{vhigh}-{vlow}_Temp{Tstart}-{Tstop}K')

### Colors Palette

In [ ]:
DataPlot.gui_pan_color()

## File Inspection

In [ ]:
list(plotobj.proj_path.glob("*"))

In [ ]:
os.startfile(plotobj.proj_path)  # open the explorer window

# Processing Template
### load sweep files
```python
gate_lst = []  # the sweeped values
gate_lst.sort()
df_lst = []
for ivg in gate_lst:
    df_loaded = plotobj.load_dfs(mod_tuple, *vars, tmpfolder="")
    df_loaded = plotobj.identify_direction(df_loaded, "B")  # add direction label
    df_loaded["gate"] = ivg
    df_lst.append(df_loaded)
overall_df = pd.concat(df_lst, ignore_index=True)
```
### calculate loop differences (e.g. for hysteresis loop)
```python
diff = plotobj.loop_diff(df_loaded, "vary-property",  "observe-property")
```

# Colormap
### Create Cmap
```python
mycmap = ListedColormap(["red", "blue", "green"])  # discrete colormap
mycmap = LinearSegmentedColormap.from_list("custom", [(0, "red"), (0.5, "blue"), (1, "green")], N = 256)  # Linear colormap (default to 256, continuous effect)
colormaps.get_cmap(mycmap)  # plot the cmap to show the effect
```
### Modify Existing Cmap
##### Reverse
reverse any existing colormap by appending `_r` to its name or using the `.reversed()` method.
##### Truncate a Cmap
use `constants.truncate_cmap(cmap, min_val, max_val, n=256)`
##### Combine Cmaps
use `constants.combine_cmap(cmap_lst, segment=128)`

#### Common Issues
`(df1[col1] == val1) & (df2[col2] == val2)`, the parentheses here are necessary

## Plotting Template

### generic
```python
# the return value of the load_dfs is exact the same as that stored in dfs[measurename_main], used for comparison or other cases
# or use df from other sources, there is no diff
df_measure_appendix = plotobj.load_dfs(measure_mods, *params,tmpfolder="",cached = False)
measurename = "VV-II-BT"
# rename columns if needed
#plotobj.rename_columns(measurename,{2:"T"})
df_measure_appendix.rename({2: "T"}, inplace=True)
plotobj.set_unit({"I":"uA"})

fig, ax, params = DataPlot.init_canvas(1,1, 10, 6, lines_per_fig=2) # params: rows x cols x 2
axtwin = ax.twinx()
# params[int, int, int] is multidimension-array-like, params[int] is flattened-array-like
params[0,0,0].update(label="")  # could also use one int to index the params

line = ax.plot(plotobj.dfs[measurename]["I"] * plotobj.unit_factor("I"),plotobj.dfs[measurename]["V"] * plotobj.unit_factor("V"),**params[0,0,0])
line2 = axtwin.plot(plotobj.dfs[measurename]["I"] * plotobj.unit_factor("I"),plotobj.dfs[measurename]["V"] * plotobj.unit_factor("V"),**params[0,0,1])

ax.set_xlabel(rf"$I$ ({plotobj.unit_name("I")})")
ax.set_ylabel(rf"$V$ ({plotobj.unit_name("V")})")
axtwin.set_ylabel(rf"$V$ ({plotobj.unit_name("V")})")
DataPlot.paint_colors_twin_axes(ax_left=ax, color_left="r", ax_right=axtwin, color_right="b")
# line is a list, so use addition to combine all of them
ax.legend(handles = line+line2, labels=[i.get_label() for i in line+line2],loc = "best", prop=DataPlot.legend_font)
ax.title.set_text(r"")
```

### For multiple lines
- use addition to add lines to a lst for legend management (`line_lst += ax.plot(...)`)
- use `cmap = LinearSegmentedColormap.from_list(...)` to get cmap, and use `cmap(idx)` to get the color
- modify the params length in `init_canvas`, and modify parameters for each line in params

### Contourf
`tricontourf` is for not aligned contour (1-D arrays)
`contourf` is for aligned contour (2-D meshes)
```python
norm = mpl.colors.Normalize(vmin=-2E-8, vmax=2E-8)  # make white to be exactly 0
tcf = ax.tricontourf(x_arr, y_arr, z_arr, levels=128, cmap="seismic", norm=norm)
cb = fig.colorbar(tcf)
cb.set_label(rf"$I$ (%)")
cb.set_ticks(np.linspace(-0.1,0.1,11))
ax.set_xlabel("B (T)")
ax.set_ylabel("G (V)")
```